<a href="https://colab.research.google.com/github/Alex64-1149/VoxNote/blob/IA-VoxNote/VoxNote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adaptation des données

In [8]:

#adaptation de https://github.com/musikalkemist/pytorchforaudio

import torch
import torchaudio
from torch.utils.data import Dataset
import pandas as pd


class ReconnaissanceVocaleDataset(Dataset) :

  #Initialise les aurguments de la classe dataset:
  # 1.fichier annotation(fichier txt qui contient tous les noms des fichiers audio (wav))
  # 2.fichier audio ( contient tous les fichiers audio(wav))
  def __init__(self,FICHIER_ANNOTE,FICHIER_AUDIO, melSpectrogram, SAMPLE_RATE, NOMBRE_ECHANTILLONS, processeur):
    self.fichierAnnotations=pd.read_csv(FICHIER_ANNOTE)
    self.fichierAudio=FICHIER_AUDIO
    self.processeur = processeur
    self.melSpectrogram = melSpectrogram.to(processeur) #s'assurer que tout se fasse au même endroit dans l'ordinateur : https://stackoverflow.com/questions/63061779/pytorch-when-do-i-need-to-use-todevice-on-a-model-or-tensor
    self.SAMPLE_RATE_VOULU = SAMPLE_RATE
    self.NOMBRE_ECHANTILLONS = NOMBRE_ECHANTILLONS

  #retourne le nombre de fichiers dans notre dataset
  def __len__(self):
    return len(self.fichierAnnotations)

  #retourne l'audio ainsi que son fichier texte associé
  def __getitem__(self, index):
    pathAudio = self.getAudioSamplePath(index)
    texte = self.getAudioSampleText(index)
    #signal informatique et sample rate de notre audio
    signal, sr = torchaudio.load(pathAudio)

    #mettre le signal au même endroit que sa transformation
    signal = signal.to(processeur)

    #normaliser l'audio
    signal = self.resampleSiNecessaire(signal, sr)
    signal = self.combinerSiNecessaire(signal)
    #diminuer ou ajouter des échantillons "vides" si le nombre d'échantillons ne correspond à 22 050
    signal = self.couperSiNecessaire(signal)
    signal = self.paddingSiNecessaire(signal)

    #transformer l'audio dans le spectogram de mel
    signal = self.melSpectrogram(signal)
    return signal, texte

  #mettre tous les fichiers audios à la même fréquence d'échantillonage
  def resampleSiNecessaire(self, signal, sr):
    if sr != self.SAMPLE_RATE_VOULU :
      resampler = torchaudio.transforms.Resample(sr, self.SAMPLE_RATE_VOULU)
      signal = resampler(signal)
    return signal

  #s'assurer que l'audio ne contient qu'une entrée et sortie (que le son ne soit pas stéréo) pour le normaliser
  def combinerSiNecessaire(self, signal):
    if signal.shape[0] > 1 :
      signal = torch.mean(signal, dim=0, keepdim= True)
    return signal

  #enlever les échantillons audios superflus
  def couperSiNecessaire(self, signal) :
    #le signal est composé de 2 dimensions : [nombre de source du signal(1 dans ce cas), longueur du signal – nombre d'échantillons (on veut 22 050)]
    if signal.shape[1] > self.NOMBRE_ECHANTILLONS :
      signal = signal[:, :self.NOMBRE_ECHANTILLONS] #utilité de [:, :] : la première dimension est prise au complet et la deuxième jusqu'à l'atteinte du nombre d'échantillon (expliquer à https://youtu.be/WyJvrzVNkOc?list=PL-wATfeyAMNoirN4idjev6aRu8ISZYVWm&t=478)
    return signal

  def paddingSiNecessaire(self, signal) :
    #la longueur du signal = signal.shape[1] comme expliqué précédemment
    if signal.shape[1] < self.NOMBRE_ECHANTILLONS :
      paddingDuSignal = (0, self.NOMBRE_ECHANTILLONS - signal.shape[1]) #(0, nombre d'échantillons manquants)
      torch.nn.functional.pad(signal, paddingDuSignal) #ajoute le padding au signal (https://pytorch.org/docs/stable/generated/torch.nn.functional.pad.html)
    return signal


  #retourne le chemin pour avoir le bon fichier audio à un certain index du FICHIER_ANNOTE
  def getAudioSamplePath(self, index):
    return self.fichierAudio[index]

  #retourne le fichier texte associé à l'audio d'un certain index du FICHIER_AUDIO
  def getAudioSampleText(self, index):
    return self.fichierAnnotations[index]


FICHIER_ANNOTE = "/content/drive/MyDrive/Colab Notebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_text.list"
FICHIER_AUDIO = "/content/drive/MyDrive/Colab Notebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_wavs.list"


#nombre d'échantillons par secondes dans notre audio
SAMPLE_RATE = 22050
NOMBRE_ECHANTILLONS = 22050

if torch.cuda.is_available(): #détermine ce qui exécute le programme (gpu préférable pour AI audio)
    processeur = "cuda"
else:
    processeur = "cpu"
print(f"Utiliation du processeur {processeur}")

#le Spectogram de Mel est une échelle logarithmique utilisée pour mieux représenter les différences qu'un humain entend dans un fichier audio ce qui aide à l'analyse sonore
melSpectrogram = torchaudio.transforms.MelSpectrogram(
    SAMPLE_RATE,
    n_fft=512, #longueur physique du signal optimale pour la reconnaissance vocale selon : https://librosa.org/doc/main/generated/librosa.stft.html
    hop_length=512, #nombre d'échantillon audio adjacents analysés par la transformée de fourier : https://librosa.org/doc/main/generated/librosa.stft.html
    n_mels=64 #nombre de séparations d'une seule fréquence optimale pour la reconnaisance vocale selon:https://stackoverflow.com/questions/62623975/why-128-mel-bands-are-used-in-mel-spectrograms
)


rvd = ReconnaissanceVocaleDataset(FICHIER_ANNOTE, FICHIER_AUDIO, melSpectrogram, SAMPLE_RATE, NOMBRE_ECHANTILLONS, processeur)


Utiliation du cuda


In [7]:
from google.colab import drive #nécessaire qu début de chaque session
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#utilisé pour extraire le zip de google drive
#from google.colab import drive
#drive.mount('/content/drive')


#!unzip file.zip


# Réseau de neurone

In [20]:
#connaissances nécessaires au CNN trouvées à l'adresse suivante : https://ketanhdoshi.github.io/Audio-ASR/
#CNN initial vient de https://www.youtube.com/watch?v=SQ1iIKs190Q&list=PL-wATfeyAMNoirN4idjev6aRu8ISZYVWm&index=8

import torch
import torchaudio
from torch import nn
!torch -m pip install torchsummary
from torchsummary import summary

class CNNNEtwork(nn.Module) :
  def __init__(self, nbCaracteresPossible) :
    super().__init__()
    outChannelsInitial = 128
    # 4 conv blocks / flatten / linear /softmax
    """
    1. Créer les 4 blocs du CNN
    2. 'flatten' le résultat en diminuant le nombre de dimensions créées avec les blocs du CNN
    3. transformer en équation linéaire les données fournies : https://docs.kanaries.net/topics/Python/nn-linear
    4. normaliser les résultats à l'aide de softmax
    """
    self.conv1 = nn.Sequential(
        nn.Conv2d( #couche en 2 dimensions
            in_channels=1, #nombre de input initial (=1 lors de l'adaptation des données)
            out_channels=outChannelsInitial, #nombre de filtre dans cette couche du réseau de neurone
            kernel_size=3, #nombre de choses analysées en même temps : https://stats.stackexchange.com/questions/296679/what-does-kernel-size-mean
            stride = 1, #déplacement du kernel : https://deepai.org/machine-learning-glossary-and-terms/stride
            padding = 2 #comme dans RSD mais pour le kernel
        ),
        nn.ReLU(), #régression linéaire
        nn.MaxPool2d(kernel_size=2) # reformulation des données : https://www.geeksforgeeks.org/apply-a-2d-max-pooling-in-pytorch/
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(
            in_channels=outChannelsInitial, # = au out_channels précédent
            out_channels=outChannelsInitial*2, # out_channels précédent *2
            kernel_size=3,
            stride = 1,
            padding = 2
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv3 = nn.Sequential(
        nn.Conv2d(
            in_channels=outChannelsInitial*2,
            out_channels=outChannelsInitial*4,
            kernel_size=3,
            stride = 1,
            padding = 2
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv4 = nn.Sequential(
        nn.Conv2d(
            in_channels=outChannelsInitial*4,
            out_channels=outChannelsInitial*8,
            kernel_size=3,
            stride = 1,
            padding = 2
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.flatten = nn.Flatten()
    self.linear = nn.Linear(outChannelsInitial*8 *5 * 4, #input (out_channels final, fréquence, temps)
                            nbCaracteresPossible #nb de output
                            )
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input_data): #envoyer les données d'une couche à une autre
    x = self.conv1(input_data)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    #passer le x des convolutional layers vers le flatten
    x = self.flatten(x)
    logits = self.linear(x) #logits signifie la probabilité (avant d'être normaliser) associée à certaines réponses : https://www.linkedin.com/posts/mwitiderrick_what-are-logits-in-deep-learning-logits-activity-7084819307959902209-UUGe#:~:text=Logits%20are%20the%20outputs%20of%20a%20neural%20network%20before%20the,belonging%20to%20a%20certain%20class.
    predictions = self.softmax(logits) #normaliser les logits
    return predictions
#algorithme pour le mapping prit sur https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
char_map_str = """
 ' 0
 <SPACE> 1
 a 2
 b 3
 c 4
 d 5
 e 6
 f 7
 g 8
 h 9
 i 10
 j 11
 k 12
 l 13
 m 14
 n 15
 o 16
 p 17
 q 18
 r 19
 s 20
 t 21
 u 22
 v 23
 w 24
 x 25
 y 26
 z 27
 - 28
 <EMP> 29
 à 30
 â 31
 ä 32
 é 33
 è 34
 ê 35
 ë 36
 î 37
 ï 38
 ô 39
 ö 40
 ù 41
 û 42
 ü 43
 ÿ 44
 ç 45
 """
#associer des caractères à des valeurs numériques
class TextTransform:
  """Maps characters to integers and vice versa"""
  def __init__(self):
      char_map_str = char_map_str
      self.char_map = {}
      self.index_map = {}
      for line in char_map_str.strip().split('\n'):
          ch, index = line.split()
          self.char_map[ch] = int(index)
          self.index_map[int(index)] = ch
      self.index_map[1] = ' '

  def text_to_int(self, text):
      """ Use a character map and convert text to an integer sequence """
      int_sequence = []
      for c in text:
          if c == ' ':
              ch = self.char_map['']
          else:
              ch = self.char_map[c]
          int_sequence.append(ch)
      return int_sequence

  def int_to_text(self, labels):
      """ Use a character map and convert integer labels to an text sequence """
      string = []
      for i in labels:
          string.append(self.index_map[i])
      return ''.join(string).replace('', ' ')

nombreCaracteres = 46

cnn = CNNNEtwork(nombreCaracteres)
summary(cnn.cuda(), (1,64,44)) #(nombre de channels, frequence, temps)

/bin/bash: line 1: torch: command not found
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 66, 46]           1,280
              ReLU-2          [-1, 128, 66, 46]               0
         MaxPool2d-3          [-1, 128, 33, 23]               0
            Conv2d-4          [-1, 256, 35, 25]         295,168
              ReLU-5          [-1, 256, 35, 25]               0
         MaxPool2d-6          [-1, 256, 17, 12]               0
            Conv2d-7          [-1, 512, 19, 14]       1,180,160
              ReLU-8          [-1, 512, 19, 14]               0
         MaxPool2d-9            [-1, 512, 9, 7]               0
           Conv2d-10          [-1, 1024, 11, 9]       4,719,616
             ReLU-11          [-1, 1024, 11, 9]               0
        MaxPool2d-12           [-1, 1024, 5, 4]               0
          Flatten-13                [-1, 20480]            

# Entraîner

# Tester


# Inférence